## Mount Google Drive

In [ ]:
from google.colab import drive

# Make sure to unmount drive at mount point
drive.flush_and_unmount()
drive.mount('/content/drive')

# Research Start

In [1]:
from datetime import datetime
from scipy import stats  
import os
import pytz
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np  

%matplotlib inline

### Creating IDX Equities Time Series

In [2]:
# idx_combined = pd.read_csv('/home/nikki/documents/amibroker/idx_exported_combined.csv', parse_dates=[1])
# idx_combined = pd.read_csv('/mnt/c/Users/nikki/Documents/data-ab/idx_exported_combined.csv', parse_dates=[1])
# idx_combined = pd.read_csv('C:/Users/nikki/Documents/data-ab/idx_exported_combined.csv', parse_dates=[1])

In [2]:
# Location of the IDX Stocks OHLC csv files
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv'
# csv_loc = '/Users/nikki/Documents/idx_exported_csv'
# csv_loc = '/home/nikki/documents/amibroker/exported-csv'
csv_loc = '/mnt/c/Users/nikki/Documents/data-ab/idx_exported_csv'
# csv_loc = 'C:/Users/nikki/Documents/data-ab/idx_exported_csv'

# Backup if the first one doesn't work
# csv_loc = '/content/drive/Shared drives/algo-clenow/idx_exported_csv_gama'

tickers = os.listdir(csv_loc)

for ticker in tickers:
    if len(ticker) != 8:
        print(f'ticker')

In [3]:
"""
Create a dictionary where the key is the ticker
and the value is a pandas dataframe of the OHLC time series
"""
data_idx = {}
for ticker in tickers:
    data_idx[ticker[:-4]] = pd.read_csv(f'{csv_loc}/{ticker}',
                                        index_col='date',
                                        parse_dates=['date'])

In [4]:
data_idx['ASII'].tail()

,open,high,low,close,volume
date,,,,,
2021-03-09,5550.0,5650.0,5400.0,5525.0,66908800
2021-03-10,5500.0,5525.0,5400.0,5425.0,66154200
2021-03-12,5550.0,5575.0,5450.0,5475.0,87065000
2021-03-15,5525.0,5625.0,5500.0,5525.0,48489500
2021-03-16,5600.0,5725.0,5550.0,5550.0,64804400


### Stocks on the Move


In [5]:
def momentum_score(ts):
    """
    Input:  Price time series.
    Output: Annualized exponential regression slope, 
            multiplied by the R2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts)) 
    # Get logs
    log_ts = np.log(ts) 
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualize percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    #Adjust for fitness
    score = annualized_slope * (r_value ** 2)
    return score

In [6]:
def volatility(ts, period=24):
    """
    Input:  Price time series, Look back period
    Output: Standard deviation of the percent change
    """
    return ts.pct_change().rolling(period).std().iloc[-1]

In [7]:
# How many (series) candles back for momentum calculation?
momentum_window = 96

# Create an empty DataFrame to store score
momentum_table = pd.DataFrame(columns=['ticker', 'score', 'vola', 'inv_vola'])

# Eliminated stocks list
eliminated_stocks = pd.DataFrame(columns=['ticker', 'score', 'vola', 'reason'])

# How many (series) candles+1 back for std dev calculation?
vola_window = 24

# How many (series) candles back for EWMA calculation?
ma_period_fast = 32
ma_period_slow = 128

# Loop the dictionary and calculate the momentum_score, then append it to pandas
for ticker, timeseries in data_idx.items():
    momentum_series = timeseries['close'].iloc[-momentum_window:]
    score = momentum_score(momentum_series)
    vola_series = timeseries['close']
    vola = volatility(vola_series, vola_window) * 16
    median_volume = timeseries['volume'].rolling(vola_window).median().iloc[-1]
    ma_fast = timeseries['close'].rolling(ma_period_fast).mean().iloc[-1]
    ma_slow = timeseries['close'].rolling(ma_period_slow).mean().iloc[-1]

#     ewma = timeseries['close'].ewm(span=ewma_period).mean().iloc[-1]

    # Need the stocks to exist at least 3 years prior (756 trading days)
    if len(timeseries) < 756:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'umur belum 3 tahun'},
                                                     ignore_index=True)
        continue
    
    # If median volume falls below 100k in the stocks, drop it 
    if median_volume < 100000:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'avg volume di bawah 100k'},
                                                     ignore_index=True)
        continue
        
    # If it has been suspended (daily vol == 0) more than once, drop it
    if timeseries['volume'].iloc[-momentum_window:].tolist().count(0) > 1:
        eliminated_stocks = eliminated_stocks.append({'ticker': ticker,
                                                      'score': score,
                                                      'vola': vola,
                                                      'reason': 'pernah disuspend lebih dari 1x'},
                                                     ignore_index=True)
        continue
    
    momentum_table = momentum_table.append({'ticker': ticker,
                                            'score': score,
                                            'vola': vola,
                                            'median_vol': median_volume,
                                            'ma_fast': ma_fast,
                                            'ma_slow': ma_slow},
                                           ignore_index=True)
    
    momentum_table['inv_vola'] = 1 / momentum_table['vola']

In [8]:
print(f'Ada {len(momentum_table)} saham lolos')
print(f'Ada {len(eliminated_stocks)} saham tereliminasi')

Ada 257 saham lolos
Ada 473 saham tereliminasi


In [9]:
momentum_table.sort_values('score', ascending=False)[:50].to_clipboard()

In [10]:
momentum_table.sort_values('score', ascending=False)[:50]

,ticker,score,vola,inv_vola,ma_fast,ma_slow,median_vol
20,ARTO,7943.274403,0.831501,1.202645,7788.09375,4103.062500,7333600.0
106,HRUM,3123.269786,0.798790,1.251893,6111.56250,3409.062500,14957000.0
205,SAME,2839.450033,0.723900,1.381406,321.03125,196.109375,41032450.0
25,ASSA,2643.508989,1.106790,0.903514,1259.06250,740.648438,29703750.0
88,EMTK,2476.031060,0.435418,2.296642,2132.81250,1322.656250,16243650.0
14,ANTM,1643.224392,0.597217,1.674433,2629.06250,1722.617188,414767552.0
231,TKIM,1375.042829,0.498031,2.007909,14792.96875,9871.875000,5649900.0
230,TINS,1258.342948,0.873287,1.145099,2050.62500,1393.398438,167500448.0
23,ASJT,1059.674831,0.872074,1.146691,236.93750,168.687500,301700.0
30,BBKP,936.808249,0.535603,1.867054,536.78125,404.093750,188672400.0


In [13]:
eliminated_stocks[eliminated_stocks['reason'] == 'umur belum 3 tahun'].sort_values('score', ascending=False)

,ticker,score,vola,reason
40,BRIS,3815.691907,9.371545e-01,umur belum 3 tahun
82,DMMX,1200.297792,6.199791e-01,umur belum 3 tahun
44,CBMF,601.234993,3.262737e-01,umur belum 3 tahun
171,SAPX,529.212255,3.554760e-01,umur belum 3 tahun
43,INDO,466.787826,4.488161e-01,umur belum 3 tahun
...,...,...,...,...
83,KBAG,-85.764465,1.415350e+00,umur belum 3 tahun
0,ITIC,-90.342123,4.416746e-01,umur belum 3 tahun
172,HRME,-91.511054,5.381667e-09,umur belum 3 tahun
209,ARKA,-92.140816,5.981414e-01,umur belum 3 tahun


In [14]:
eliminated_stocks[eliminated_stocks['reason'] == 'pernah disuspend lebih dari 1x'].sort_values('score', ascending=False)

,ticker,score,vola,reason
163,TIFA,779.395081,0.305628,pernah disuspend lebih dari 1x
261,INCI,272.108060,0.410409,pernah disuspend lebih dari 1x
128,RANC,100.071218,0.348120,pernah disuspend lebih dari 1x
39,RMBA,74.438505,1.595118,pernah disuspend lebih dari 1x
307,RUIS,42.147570,0.923970,pernah disuspend lebih dari 1x
488,BIPP,39.918142,0.606751,pernah disuspend lebih dari 1x
109,WINS,35.294318,0.213005,pernah disuspend lebih dari 1x
60,LPPS,15.956782,0.699902,pernah disuspend lebih dari 1x
218,GWSA,13.444831,0.528288,pernah disuspend lebih dari 1x
34,AISA,8.955749,1.638821,pernah disuspend lebih dari 1x


In [15]:
eliminated_stocks[eliminated_stocks['reason'] == 'avg volume di bawah 100k'].sort_values('score', ascending=False)

,ticker,score,vola,reason
77,ARGO,489.785755,1.123822e+00,avg volume di bawah 100k
159,PTIS,358.443206,9.486231e-01,avg volume di bawah 100k
408,BBHI,323.138029,5.734052e-01,avg volume di bawah 100k
436,MERK,306.264092,4.505298e-01,avg volume di bawah 100k
358,PNSE,165.606997,3.973270e-07,avg volume di bawah 100k
...,...,...,...,...
70,PGLI,-66.517070,1.117693e+00,avg volume di bawah 100k
377,NASA,-68.554597,2.249972e-08,avg volume di bawah 100k
419,TARA,-71.126410,3.709058e-08,avg volume di bawah 100k
373,GLOB,-79.339016,5.216332e-01,avg volume di bawah 100k
